<a href="https://colab.research.google.com/github/adrien-simard/entreprise-france/blob/main/database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dowload Files 

In [ ]:
from six.moves import urllib

DOWNLOAD_URL = "https://static.data.gouv.fr/resources/liste-des-conventions-collectives-par-entreprise-siret/20220201-110309/transsismmo-weez-idcc-1221.csv"
DOWNLOAD_PATH ="idcc.csv" 
urllib.request.urlretrieve(DOWNLOAD_URL,DOWNLOAD_PATH)

('idcc.csv', <http.client.HTTPMessage at 0x7f8b4a4d8a10>)

In [ ]:
from six.moves import urllib

DOWNLOAD_URL = "https://static.data.gouv.fr/resources/liste-des-conventions-collectives-par-entreprise-siret/20220201-110309/transsismmo-weez-idcc-1221.csv"
DOWNLOAD_PATH ="idcc.csv" 
urllib.request.urlretrieve(DOWNLOAD_URL,DOWNLOAD_PATH)

('idcc.csv', <http.client.HTTPMessage at 0x7f8b4a06e1d0>)

In [ ]:

DOWNLOAD_URL = "https://files.data.gouv.fr/geo-sirene/last/StockEtablissement_utf8_geo.csv.gz"
DOWNLOAD_PATH ="StockEtablissement_utf8_geo.csv.gz" 
urllib.request.urlretrieve(DOWNLOAD_URL,DOWNLOAD_PATH)

('StockEtablissement_utf8_geo.csv.gz',
 <http.client.HTTPMessage at 0x7f8b4a06c1d0>)

In [ ]:
DOWNLOAD_URL = "https://www.data.gouv.fr/fr/datasets/r/825f4199-cadd-486c-ac46-a65a8ea1a047"
DOWNLOAD_PATH ="siren.zip" 
urllib.request.urlretrieve(DOWNLOAD_URL,DOWNLOAD_PATH)

('siren.zip', <http.client.HTTPMessage at 0x7f8b49fa8310>)


##Data processing


In [ ]:

import zipfile
with zipfile.ZipFile("siren.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 58.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ed5603a7a1d75d1f1203f0cf93421a33af893a1a83785d02dacff84604fa9b2c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,BooleanType,DateType

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark app") \
    .getOrCreate()

In [ ]:
import pandas as pd
df = pd.read_csv('targetdir/StockUniteLegale_utf8.csv',nrows=1000)

In [ ]:
idcc = spark.read.csv("idcc.csv",header=True)
idcc = idcc.withColumnRenamed("SIRET","siret_idcc")

In [ ]:
siren = spark.read.csv('targetdir/StockUniteLegale_utf8.csv',header=True)

In [ ]:
sirengeo = spark.read.csv("StockEtablissement_utf8_geo.csv.gz",header=True)
sirengeo = sirengeo.withColumnRenamed("siren","siren_geo")

In [ ]:
siren = siren.select("siren","denominationUniteLegale","nomUniteLegale","nomUsageUniteLegale","categorieJuridiqueUniteLegale","activitePrincipaleUniteLegale")

In [ ]:
sirengeo = sirengeo.select("siren_geo","siret","dateCreationEtablissement","trancheEffectifsEtablissement","geo_adresse","codePostalEtablissement")

In [ ]:
employeurGeo = sirengeo.join(siren, sirengeo.siren_geo == siren.siren, 'inner').drop("siren_geo")

In [ ]:
merged = employeurGeo.join(idcc, employeurGeo.siret == idcc.siret_idcc, how='inner').drop("siret_idcc")

In [ ]:
#merged = merged.select("nomUniteLegale","siren","siret","statutDiffusionEtablissement","dateCreationEtablissement","trancheEffectifsEtablissement","etablissementSiege","nomUsageUniteLegale","categorieJuridiqueUniteLegale","activitePrincipaleUniteLegale","geo_adresse","longitude","latitude","codePostalEtablissement","IDCC")
merged.na.drop(subset="activitePrincipaleUniteLegale")
merged = merged.dropDuplicates(["siret"])

In [ ]:
from pyspark.sql.functions import concat,col
from pyspark.sql.functions import lit

merged = merged.withColumn("contact",lit("https://www.pagesjaunes.fr/siret/"))
#merged = merged.withColumn("contact",concat(merged.contact,merged.siret))
merged = merged.select(concat(merged.contact,merged.siret)
              .alias("contact"),"siren","denominationUniteLegale","siret","dateCreationEtablissement","trancheEffectifsEtablissement","nomUsageUniteLegale","categorieJuridiqueUniteLegale","activitePrincipaleUniteLegale","geo_adresse","codePostalEtablissement","IDCC")


In [ ]:
opco = spark.read.csv("opco.csv",sep=";",header=True)
opco = opco.withColumnRenamed("IDCC","IDCC_opco")
opco = opco.withColumn("IDCC_opco",opco.IDCC_opco.cast('integer'))
merged = merged.withColumn("IDCC",merged.IDCC.cast('integer'))

In [ ]:
merged = merged.join(opco, merged.IDCC == opco.IDCC_opco, how='inner').drop("IDCC_opco").drop("obs")

In [ ]:
merged.show()

+--------------------+---------+--------------+--------------+----------------------------+-------------------------+-----------------------------+------------------+-------------------+-----------------------------+-----------------------------+--------------------+---------+---------+-----------------------+----+------------------------+--------------------+---------+--------------+-------------------+------------------------+--------------------+
|             contact|    siren|nomUniteLegale|         siret|statutDiffusionEtablissement|dateCreationEtablissement|trancheEffectifsEtablissement|etablissementSiege|nomUsageUniteLegale|categorieJuridiqueUniteLegale|activitePrincipaleUniteLegale|         geo_adresse|longitude| latitude|codePostalEtablissement|IDCC|Operateur de competences|             Libelle|siren_nom|nomUniteLegale|nomUsageUniteLegale|Operateur de competences|             Libelle|
+--------------------+---------+--------------+--------------+----------------------------+-

In [ ]:
merged.repartition(1).write.csv('/content/drive/MyDrive/FI5/sirenpreprocessed1.csv',header=True)